# To build a recommender system that matches users for a trade based on their wanted and selling list

In [1]:
#  %cd /content/drive/MyDrive/CMPE_256_Project
#  ! git clone https://github.com/swapit/code.git

## Required Imports

In [2]:
import numpy as np
import sklearn as sk
import pandas as pd
from sklearn.model_selection import train_test_split
import random

/home/015266823/venv/lib/python3.6/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
print(pd.__version__)

1.1.5


# Preprocessing

In [4]:
DATA_PATH = "/home/015266823/bookmooch/bm1y"

def read_data_into_dfs(path):
  have_df = pd.read_csv(f"{DATA_PATH}/have.csv", names=["user_id", "item_id", "unix_timestamp"])
  wish_df = pd.read_csv(f"{DATA_PATH}/wish.csv", names=["user_id", "item_id", "unix_timestamp"])
  pairs_df = pd.read_csv(f"{DATA_PATH}/pairs.csv", names=["user1_id", "item1_id", "user2_id", "item2_id"])
  transac_df = pd.read_csv(f"{DATA_PATH}/transac.csv", names=["giver_user_id", "receiver_user_id", "item_id", "unix_timestamp"])
  return have_df, wish_df, pairs_df, transac_df

have_df, wish_df, pairs_df, transac_df = read_data_into_dfs(DATA_PATH)

In [5]:
have_df.head(1)

,user_id,item_id,unix_timestamp
0,41,415868,1421536572


In [6]:
wish_df.head(1)

,user_id,item_id,unix_timestamp
0,41,12889,1425062294


In [7]:
pairs_df.head(1)

,user1_id,item1_id,user2_id,item2_id
0,80555,84914,5616,495048


In [8]:
transac_df.head(1)

,giver_user_id,receiver_user_id,item_id,unix_timestamp
0,38,15502,354627,1416308635


### Prepare User-Item Matrix

In [9]:
def user_item_df(have_df, wish_df, pairs_df, transac_df):
  # Grab all the items the user has wished for
  wished_for = wish_df[['user_id',	'item_id']]

  # Get items from the transactions.
  transacted = transac_df[['receiver_user_id', 'item_id']].rename({'receiver_user_id': 'user_id'}, axis='columns')

  # Combine the wished for and transaction dataframe
  user_item = pd.concat([wished_for, transacted])

  # Add a like column indicating they have liked it.
  user_item['like'] = 1

  # These are the duplicate entries (that is, when someone got something they wished for)
  print(f'Number of user_items before removing duplicates: {user_item.shape[0]}')
  print(f'Number of duplicated rows: {user_item[user_item.duplicated()].shape[0]}')
  user_item = user_item.drop_duplicates()
  print(f'Number of duplicated rows after removing: {user_item[user_item.duplicated()].shape[0]}')
  print(f'Number of user_items after removing duplicates: {user_item.shape[0]}')

  return user_item

In [10]:

user_item = user_item_df(have_df, wish_df, pairs_df, transac_df)

Number of user_items before removing duplicates: 546785
Number of duplicated rows: 8613
Number of duplicated rows after removing: 0
Number of user_items after removing duplicates: 538172


In [11]:
user_item.user_id.describe()

count    538172.000000
mean      43788.941558
std       26092.649412
min          41.000000
25%       21341.000000
50%       44447.000000
75%       66695.000000
max      124671.000000
Name: user_id, dtype: float64

### Change Ids To Sequence

In [12]:
def change_ids_to_seq(user_item):
  # Create user and item mapping to convert to seq
  user_ids = user_item['user_id'].unique()
  user_ids.sort()

  item_ids = user_item['item_id'].unique()
  item_ids.sort()

  user_id_map = {real: mapped for mapped, real in enumerate(user_ids)}
  item_id_map = {real: mapped for mapped, real in enumerate(item_ids)}

  def find_user_id_mapping(user_id, user_id_map):
    return user_id_map[user_id]
  
  def find_item_id_mapping(item_id, item_id_map):
    return item_id_map[item_id]

  user_item["user_id"] = user_item.apply(lambda s: find_user_id_mapping(s["user_id"], user_id_map), axis=1)
  user_item["item_id"] = user_item.apply(lambda s: find_item_id_mapping(s["item_id"], item_id_map), axis=1)
  return user_item, user_id_map, item_id_map

In [13]:
user_item, user_id_map, item_id_map = change_ids_to_seq(user_item)

### Train-Test Split

In [14]:
def split_into_test_train(user_item):
  train_df, test_df = train_test_split(user_item, test_size=0.2)

  test_df_zreos = test_df.copy()
  test_df_zreos["like"] = 0
  train_df = train_df.append(test_df_zreos)
  # train_df[train_df["like"] == 0]

  return train_df, test_df

In [15]:
train_df, test_df = split_into_test_train(user_item)

In [16]:
train_df

,user_id,item_id,like
244918,7181,170092,1
289696,8343,204289,1
218390,6371,178162,1
260569,7639,4343,1
208198,6157,32132,1
...,...,...,...
191949,5571,50413,0
258054,7604,192515,0
267994,7736,111646,0
250909,7370,26299,0


In [17]:
test_df

,user_id,item_id,like
134845,3642,59431,1
49382,1170,12795,1
291752,8425,97971,1
68307,1835,95508,1
260671,7647,111724,1
...,...,...,...
191949,5571,50413,1
258054,7604,192515,1
267994,7736,111646,1
250909,7370,26299,1


# Modeling Basic User Preferences





In [18]:
def get_train_matrix(train_df):
  # Get the training matrix with 0s entered for all user-item combos.
  train_matrix_df = train_df.pivot(index='user_id', columns ='item_id', values = 'like').fillna(0)
  return train_matrix_df

In [19]:
matrix_df = get_train_matrix(train_df)

In [20]:
matrix_df

item_id,0,1,2,3,4,5,6,7,8,9,...,207320,207321,207322,207323,207324,207325,207326,207327,207328,207329
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8613,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8614,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8615,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
matrix = matrix_df.to_numpy()

In [22]:
matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 1., 1.]])

The first goal of our model is to estimate a user’s preference for an individual item. As our data contains wish lists
and past transactions, we use them as implicit feedback signals [18] when building the preference model.


Following the approach proposed by Hu et al. [10], the
user-item interaction matrix R is built based on implicit
feedback signals as follows:

(i.e., 1 iff the item belongs to uj ’s wish list, or there exists
a past transaction in which uj receives ik).

where puj and qik
are vectors describing the ‘preferences’ of
the user uj and the ‘properties’ of the item ik.


## Social Bias

#### Map Social friendship/Trust between users using their Transaction History

In [26]:
#calculating total number of transactions between user1 and user2

#Using Dict, key=(giver_user_id,receiver_user_id) ,value = total number of transactions between giver_user_id and receiver_user_id .
#return: It is returning dict which contains all the number of transactions between giver and reciever. 


def transactions_U1_U2(transac_df):
  
  socialfactorU1_U2_Dict={}
  
  for i,row in transac_df.iterrows():
    # print(i['giver_user_id'])
    key=(row['giver_user_id'],row['receiver_user_id'])
    if key in socialfactorU1_U2_Dict:
      
      socialfactorU1_U2_Dict[key]=socialfactorU1_U2_Dict[key]+1
    else:
      
      socialfactorU1_U2_Dict[key]=1
  # print(tempkey)
  return socialfactorU1_U2_Dict

In [27]:
socialfactorU1_U2_Dict=transactions_U1_U2(transac_df)

In [28]:
len(socialfactorU1_U2_Dict)

109628

#### Map All the users in Have_df to retrieve the users having a particular Item Id

In [29]:
#In this we are retrieving a list of all the users having a specific item in the form of {"itemid":["user1","user2"],"itemid2":["user3"] }from have_df.
#after this we will call recommend a user from above list who is having that item and high social bias score for that particular user.

#haveDict is the dictionary for all the users having that item
def retrieveAllUsersHavingThatItem(have_df):
    
    haveDict = {}

    tempKey = 0
    for i,row in have_df.iterrows():
        # print(i['giver_user_id'])
        value=row['user_id']
        key=row['item_id']
        if key in haveDict:
            if value not in haveDict[key]:
                haveDict[key].append(value)
        else:
            haveDict[key] = [value]
    return haveDict

In [30]:
UsersHavingThatItem_map=retrieveAllUsersHavingThatItem(have_df)

In [31]:
len(UsersHavingThatItem_map)

65678

#### SocialActivity Factor counts the number of transactions an individual User has done based on his past transactions

In [32]:
#Calculating the value of transactionCountFactor to check if the user is socially Active?
#Adding the sociallyActiveUserFactor to the result of social bias to get the socialScore


#Transaction Count 
def getSocialActivityFactor(transac_df,pairs_df):
    transactionCount = {}
    max_variable = 0
    #Transactions from transac_df
    for i,row in transac_df.iterrows():
        # print(i['giver_user_id'])
        key1 = row['giver_user_id']
        key2 = row['receiver_user_id']

        if key1 in transactionCount:
            transactionCount[key1] = transactionCount[key1]+1
            max_variable = max(max_variable,transactionCount[key1])
        else:
            transactionCount[key1] = 1

        if key2 in transactionCount:
            transactionCount[key2] = transactionCount[key2]+1
            max_variable = max(max_variable,transactionCount[key2])
        else:
            transactionCount[key2] = 1
            
    #Transactions from pairs_df
    for i,row in pairs_df.iterrows():
        # print(i['giver_user_id'])
        key1 = row['user1_id']
        key2 = row['user2_id']

        if key1 in transactionCount:
            transactionCount[key1] = transactionCount[key1]
            max_variable = max(max_variable,transactionCount[key1])
        else:
            transactionCount[key1] = 1
        if key2 in transactionCount:
            transactionCount[key2] = transactionCount[key2]
            max_variable = max(max_variable,transactionCount[key2])
        else:
            transactionCount[key2] = 1
#     print (max_variable) 
    return transactionCount,max_variable

In [33]:
socialActivityDict,maxTransaction= getSocialActivityFactor(transac_df,pairs_df)

#### Social Bias Score gives the normalised score based on the User Activity and User-User Transactions

In [34]:
def socialBiasScore(userId,itemId):

    giverId = 0
    maximumScore = 0
    if itemId in UsersHavingThatItem_map:
        for i in UsersHavingThatItem_map[itemId]:
            try:
                maximumScore = max(maximumScore,socialfactorU1_U2_Dict[i][userId])
                
                giverId = i
            except:
                try:
                    maximumScore = max(maximumScore,socialfactorU1_U2_Dict[userId][i])
                    giverId = i
                except:
                    pass
    
    
    if maximumScore == 0:
        if userId in socialActivityDict:
            return (socialActivityDict[userId]/len(socialActivityDict))
        
        return 0
    

    social_bias_score=  (maximumScore/maxTransaction)    
    #print(maxTransaction)   
    return social_bias_score

# Evaluation 

In [35]:
def print_full(x):
  pd.set_option('display.max_rows', len(x))
#   print(x)
  pd.reset_option('display.max_rows')


def get_test_test_triplates(predicted_user_items_df, test_df, user_item, user_id_map, item_id_map,socialBias):

  def find_predicted_for_liked_item(user_id, item_id):
    
    socialBiasValue = socialBiasScore(user_id,item_id)
   
    if socialBias:
        return predicted_user_items_df.iloc[user_id][item_id].item() + socialBiasValue
        #return sum(predicted_user_items_df.iloc[user_id][item_id].item() ,float(socialBiasScore(user_id,item_id,have_df,socialBiasFactorDict)))
    else:
        return predicted_user_items_df.iloc[user_id][item_id].item()
        

  def get_random_item_for_user(user_id):
    random_item_id = random.randrange(0, len(item_id_map)) 

    liked_item_ids_for_user = set(user_item[user_item["user_id"] == user_id]["item_id"].values)
    if not random_item_id in liked_item_ids_for_user:
      return random_item_id

    while random_item_id in liked_item_ids_for_user:
      random_item_id = random.randrange(0, len(item_id_map))
    return random_item_id

  def find_predicted_for_random_item(user_id, item_id):
#     print(user_id)
#     print(item_id)
    return predicted_user_items_df.iloc[int(user_id)][int(item_id)].item()

  
  evaluation_df = test_df[["user_id",	"item_id"]].rename({'item_id': 'liked_item_id'}, axis='columns')
  evaluation_df["prediction_for_liked"] = evaluation_df.apply(lambda s: find_predicted_for_liked_item(s["user_id"], s["liked_item_id"]), axis=1)
  evaluation_df["random_item_id"] = evaluation_df.apply(lambda s: get_random_item_for_user(s["user_id"]), axis=1)
  evaluation_df["prediction_for_random"] = evaluation_df.apply(lambda s: find_predicted_for_random_item(s["user_id"], s["random_item_id"]), axis=1)

  return evaluation_df

In [39]:
def evaluation_func(evaluation_df):
  def get_difference(liked_item_pred, random_item_pred):
    diff = liked_item_pred - random_item_pred
    if diff > 0:
      return 1
    else:
      return 0
  evaluation_df["difference"] = evaluation_df.apply(lambda s: get_difference(s["prediction_for_liked"], s["prediction_for_random"]), axis=1)
  return evaluation_df["difference"].sum()/evaluation_df["difference"].count()

# SVD and Results Of Evaluation with and without Social Factor

In [40]:
from scipy.sparse.linalg import svds


for n in range(50, 500, 50):
    
    print(f"N: {n}")
    U, sigma, Vt = svds(matrix, k = n)
    sigma = np.diag(sigma)


    predicted_user_items = np.dot(np.dot(U, sigma), Vt)
    predicted_user_items_df = pd.DataFrame(predicted_user_items, columns = matrix_df.columns)
    predicted_user_items_df = predicted_user_items_df.rename_axis('user_id').rename_axis("item_id", axis="columns")

    evaluation_df = get_test_test_triplates(predicted_user_items_df, test_df, user_item, user_id_map, item_id_map,False)
    e = evaluation_func(evaluation_df)
    print(f"Social Bias: False E: {e}")

    evaluation_df_social = get_test_test_triplates(predicted_user_items_df, test_df, user_item, user_id_map, item_id_map,True)
    e = evaluation_func(evaluation_df_social)
 
    print(f"Social Bias: True E: {e}")

N: 50
Social Bias: False E: 0.5434833333333333
Social Bias: True E: 0.6154833333333334
N: 100
Social Bias: False E: 0.5713166666666667
Social Bias: True E: 0.6355333333333333
N: 150
Social Bias: False E: 0.5752
Social Bias: True E: 0.63785
N: 200
Social Bias: False E: 0.5646166666666667
Social Bias: True E: 0.6283333333333333
N: 250
Social Bias: False E: 0.5546166666666666
Social Bias: True E: 0.6216333333333334
N: 300
Social Bias: False E: 0.5400833333333334
Social Bias: True E: 0.6108833333333333
N: 350
Social Bias: False E: 0.53425
Social Bias: True E: 0.6036
N: 400
Social Bias: False E: 0.5240333333333334
Social Bias: True E: 0.5976333333333333
N: 450
Social Bias: False E: 0.5171833333333333
Social Bias: True E: 0.5867833333333333
